In [1]:
import pandas as pd
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.prompts import ChatPromptTemplate
# from llm_client_chat import AlpacaLLM
from llm_client import AlpacaLLM

import time, os
os.environ['HF_HOME'] = './cache/'

In [2]:
def load_embedding_model(model_path : str):
    start_time = time.time()
    encode_kwargs = {"normalize_embeddings": True}
    local_embedding = HuggingFaceEmbeddings(
        model_name=model_path,
        cache_folder="./models",
        encode_kwargs=encode_kwargs
    )
    end_time = time.time()
    print(f'model load time {round(end_time - start_time, 0)} second')
    return local_embedding

embedding = load_embedding_model(model_path="intfloat/multilingual-e5-large")

d:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


model load time 27.0 second


In [3]:
reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

In [4]:
excel_pertanyaan = pd.read_excel("Pertanyaan Benchmark.xlsx", sheet_name=None, index_col=0)

In [15]:
# CHROMA_PATH = "final_test/unstructured-multi"
CHROMA_PATH = "final_test/langchain-multi"
# ==================================
embed_model = "Multilingual-e5-large"
# ==================================
# loader = "Unstructured"
loader = "langchain"
# ==================================
reranker = "bge-reranker-v2-m3"
# reranker = "ms-marco-MultiBERT-L-12"
# reranker = "None"

In [6]:
PROMPT_TEMPLATE = """### Instruction:
Your job is to answer the question based on the given pieces of information. All you have to do is answer the question. Not all of the information provided may be relevant to the question. Each piece of information will be separated by '---'.

### Example:
Question: Apa manfaat olahraga teratur bagi kesehatan kardiovaskular?
---

Research published in the Journal of the American Heart Association indicates that regular exercise can reduce the risk of coronary heart disease by up to 30%. Physical activity helps strengthen heart muscles, improve blood circulation, and lower blood pressure.

---

Although exercise has many benefits, it is important to do it correctly to avoid injuries. Warming up before exercising and cooling down afterwards are highly recommended. Additionally, the type of exercise chosen should match the individual's physical condition to avoid unwanted risks.

---

According to a study from the Mayo Clinic, people who exercise regularly have better cholesterol levels and tend to have a healthier weight. Exercise can also increase insulin sensitivity and help regulate blood sugar levels, which are important factors in maintaining heart health.

---

Answer:
Regular physical exercise has several benefits for cardiovascular health. Firstly, it can reduce the risk of coronary heart disease by up to 30%, as it strengthens the heart muscles, improves blood circulation, and lowers blood pressure. Secondly, individuals who exercise regularly tend to have better cholesterol levels and a healthier weight, which are crucial for heart health. Additionally, regular exercise can increase insulin sensitivity and help regulate blood sugar levels, further contributing to cardiovascular well-being.

### Another example:
Question: Apa manfaat diet kaya serat untuk kesehatan pencernaan?
---

A fiber-rich diet is known to prevent constipation by increasing stool bulk and softness, making it easier to pass. Fiber also helps maintain gut health by promoting the growth of beneficial bacteria in the digestive system.

---

High-fiber foods such as fruits, vegetables, and whole grains are not only good for digestion but can also help control blood sugar levels and lower cholesterol. Soluble fiber in these foods helps slow down sugar absorption and binds cholesterol in the intestines.

---

Some studies suggest that a high-fiber diet can reduce the risk of colorectal cancer. Fiber helps speed up the elimination of carcinogenic substances from the colon, reducing the exposure time of colon cells to harmful materials.

---

Answer:
A diet rich in fiber has multiple benefits for digestive health. It can prevent constipation by increasing stool bulk and softness, making it easier to pass. Fiber also promotes gut health by encouraging the growth of beneficial bacteria in the digestive system. Additionally, high-fiber foods such as fruits, vegetables, and whole grains can help control blood sugar levels and lower cholesterol. Soluble fiber in these foods slows sugar absorption and binds cholesterol in the intestines. Furthermore, a high-fiber diet can reduce the risk of colorectal cancer by speeding up the removal of carcinogenic substances from the colon, thereby reducing the exposure time of colon cells to harmful materials.

### Input
Question: {question}
---

{context}

---
"""

In [16]:
# run without reranking
from deep_translator import GoogleTranslator

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

for sheet in excel_pertanyaan:
    jawaban = []
    generated_response = []
    for num, pertanyaan in enumerate(excel_pertanyaan[sheet]["Pertanyaan"]):
        if type(pertanyaan) == str:
            results = db.similarity_search_with_relevance_scores(pertanyaan, k=3)
            if len(results) == 0 or results[0][1] < 0.7:
                print(f"Similarity too low.", end="\n")

            context_text = "\n\n---\n\n".join([GoogleTranslator(source='id', target='en').translate(doc.page_content) for doc, _score in results])
            prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
            prompt = prompt_template.format(context=context_text, question=pertanyaan)
            model = AlpacaLLM()
            response_text = model.invoke(prompt)
            generated_response.append(response_text)

            print(prompt)
            print("Jawaban ", response_text)

            list_jwbn = []
            for doc, _score in results:
                string = (f"{doc.page_content}"
                f"\nmetadata: {doc.metadata}"
                f"\nscore: {_score}")
                list_jwbn.append(string)

            jawaban.append(list_jwbn)
        else:
            jawaban.append([pertanyaan]*3)
            generated_response.append(pertanyaan)

    jawaban1 = [doc[0] for doc in jawaban]
    jawaban2 = [doc[1] for doc in jawaban]
    jawaban3 = [doc[2] for doc in jawaban]
    # jawaban4 = [doc[3] for doc in jawaban]
    # jawaban5 = [doc[4] for doc in jawaban]

    excel_pertanyaan[sheet]["Dokumen 1"] = jawaban1
    excel_pertanyaan[sheet]["Dokumen 2"] = jawaban2
    excel_pertanyaan[sheet]["Dokumen 3"] = jawaban3
    excel_pertanyaan[sheet]["Generated Answer"] = generated_response
    # excel_pertanyaan[sheet]["Dokumen 4"] = jawaban4
    # excel_pertanyaan[sheet]["Dokumen 5"] = jawaban5

Human: ### Instruction:
your job is to answer question based on a given pieces of information. all you have to do is answer the question. not all of the information provided maybe relevant to the question. each pieces of information will be separated by '---'

### Example:
Question: Apa manfaat olahraga teratur bagi kesehatan kardiovaskular?
---

Penelitian yang dipublikasikan dalam Journal of the American Heart Association menunjukkan bahwa olahraga teratur dapat mengurangi risiko penyakit jantung koroner hingga 30%. Aktivitas fisik membantu memperkuat otot jantung, meningkatkan sirkulasi darah, dan menurunkan tekanan darah.

---

Meskipun olahraga memiliki banyak manfaat, penting untuk dilakukan dengan benar untuk menghindari cedera. Pemanasan sebelum berolahraga dan pendinginan setelahnya sangat dianjurkan. Selain itu, jenis olahraga yang dipilih harus sesuai dengan kondisi fisik individu untuk menghindari risiko yang tidak diinginkan.

---

Menurut sebuah studi dari Mayo Clinic, or

In [17]:
with pd.ExcelWriter(f'pertanyaan benchmark_{embed_model}_{loader}_None.xlsx') as writer:  
    excel_pertanyaan["PNPK"].to_excel(writer, sheet_name='PNPK')
    excel_pertanyaan["Nutrisi"].to_excel(writer, sheet_name='Nutrisi')
    excel_pertanyaan["tatalaksana pada anak"].to_excel(writer, sheet_name='tatalaksana pada anak')
    excel_pertanyaan["tatalaksana anak dan bayi"].to_excel(writer, sheet_name='tatalaksana anak dan bayi')

In [18]:
## Run with Reranking
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker, FlashrankRerank
from deep_translator import GoogleTranslator

retriever = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding).as_retriever(search_kwargs={"k": 20})

# lst = ["tatalaksana pada anak"]
for sheet in excel_pertanyaan:
    jawaban = []
    generated_response = []
    for num, pertanyaan in enumerate(excel_pertanyaan[sheet]["Pertanyaan"]):
        if type(pertanyaan) == str:
            if "bge" in reranker:
                compressor = CrossEncoderReranker(model=reranker_model, top_n=3)
                compression_retriever = ContextualCompressionRetriever(
                    base_compressor=compressor, base_retriever=retriever
                )
            elif "marco" in reranker:
                compressor = FlashrankRerank(model="ms-marco-MultiBERT-L-12", top_n=3)
                compression_retriever = ContextualCompressionRetriever(
                    base_compressor=compressor, base_retriever=retriever
                )
            results = compression_retriever.invoke(pertanyaan)
            
            context_text = "\n\n---\n\n".join([GoogleTranslator(source='id', target='en').translate(doc.page_content) for doc in results])
            prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
            prompt = prompt_template.format(context=context_text, question=pertanyaan)
            model = AlpacaLLM()
            response_text = model.invoke(prompt)
            generated_response.append(response_text)
            print(prompt)
            print("Jawaban ", response_text)
            list_jwbn = []
            for doc in results:
                string = (f"{doc.page_content}"
                f"\nmetadata: {doc.metadata}")
                list_jwbn.append(string)

            jawaban.append(list_jwbn)
        else:
            jawaban.append([pertanyaan]*3)
            generated_response.append(pertanyaan)

    
    jawaban1 = [doc[0] for doc in jawaban]
    jawaban2 = [doc[1] for doc in jawaban]
    jawaban3 = [doc[2] for doc in jawaban]
    # jawaban4 = [doc[3] for doc in jawaban]
    # jawaban5 = [doc[4] for doc in jawaban]

    excel_pertanyaan[sheet]["Dokumen 1"] = jawaban1
    excel_pertanyaan[sheet]["Dokumen 2"] = jawaban2
    excel_pertanyaan[sheet]["Dokumen 3"] = jawaban3
    excel_pertanyaan[sheet]["Generated Answer"] = generated_response
    # excel_pertanyaan[sheet]["Dokumen 4"] = jawaban4
    # excel_pertanyaan[sheet]["Dokumen 5"] = jawaban5

Human: ### Instruction:
your job is to answer question based on a given pieces of information. all you have to do is answer the question. not all of the information provided maybe relevant to the question. each pieces of information will be separated by '---'

### Example:
Question: Apa manfaat olahraga teratur bagi kesehatan kardiovaskular?
---

Penelitian yang dipublikasikan dalam Journal of the American Heart Association menunjukkan bahwa olahraga teratur dapat mengurangi risiko penyakit jantung koroner hingga 30%. Aktivitas fisik membantu memperkuat otot jantung, meningkatkan sirkulasi darah, dan menurunkan tekanan darah.

---

Meskipun olahraga memiliki banyak manfaat, penting untuk dilakukan dengan benar untuk menghindari cedera. Pemanasan sebelum berolahraga dan pendinginan setelahnya sangat dianjurkan. Selain itu, jenis olahraga yang dipilih harus sesuai dengan kondisi fisik individu untuk menghindari risiko yang tidak diinginkan.

---

Menurut sebuah studi dari Mayo Clinic, or

In [19]:
excel_pertanyaan["PNPK"]

,Pertanyaan,Jawaban,Dokumen,tipe sumber,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa penyebab kerusakan ginjal pada anak?,Penyebab utama penyakit ginjal tahap akhir (PG...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,abnormalitas struktur atau fungsi ginjal denga...,ABSTRAK Penyakit ginjal kronik (PGK) merupakan...,beberapa faktor yang menyebabkan kerusakan\nbe...,\nApa penyebab kerusakan ginjal pada anak adal...
2,Apa yang harus dijelaskan saat melakukan eduka...,1. Penjelasan mengenai perjalanan alamiah peny...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,status metabolik yang lebih baik saat inisiasi...,mendapatkan pengelolaan standar sedangkan pada...,Pasien PGTA harus diberikan informasi lengk...,Question: Apa yang harus dijelaskan saat melak...
3,Apa yang harus dipersiapkan pada pasien/ kelua...,1. Informed consent setelah mendapatkan penjel...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,c)Gangguan pertumbuhan dan perkembangan\nd) Ko...,n)Untuk mempertahankan tekanan darah dapat\ndi...,(b) Sementara tujuan terapi DP untuk \nmengopt...,### Response:\nBefore performing hemodialysis ...
4,Apa saja komplikasi akut yang dapat terjadi sa...,1. Hipotensi intradialisis\n2. kram otot\n3. g...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,c)Gangguan pertumbuhan dan perkembangan\nd) Ko...,antibiotika empiris sampai ada hasil kultur da...,Intralipid 20%). \n \n7) Komplikasi Akut Hemod...,\nbe administered intravenously if necessary. ...
5,Apa langkah pertama dalam deteksi dini Penyaki...,Langkah pertama dalam proses deteksi dini PGK ...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,"masing-masing 3,1; 15,7; 2,4; dan 41, untuk te...","dan meliputi 3 aspek utama: primer, sekunder, ...",PENDAHULUAN Penyakit ginjal kronik (PGK) atau ...,\nCONCLUSIONS
6,Apa syarat untuk memasang kembali kateter hemo...,"tidak demam selama 48-72 jam, kultur darah neg...",Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,Vankomisin intravena (pada infeksi MRSA atau C...,kultur darah negatif 2 kali berturutan; 3) kad...,dilakukan lebih sering. \n7) Pengambilan sam...,### Response:\nFor a hemodialysis catheter to ...
7,Apa tujuan dari terapi nutrisi perioperatif?,Tujuan terapi nutrisi adalah memperbaiki statu...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,setidaknya satu dari kriteria dibawah ini: \n...,[Peringkat bukti: Level III] \nDari hasil pene...,"yang tidak adekuat, inhibisi pelepasan besi d...",\nAnswer:\nThe purpose of perioperative nutrit...
8,Apa saja kriteria yang menunjukkan pasien beda...,Kriteria tersebut meliputi penurunan berat bad...,Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,f. Status volume cairan Pasien PGK berisiko u...,setidaknya satu dari kriteria dibawah ini: \n...,[Peringkat bukti: Level III] \nDari hasil pene...,\nJawaban:\nThe following are indicators showi...
9,Apa saja efek samping hemodialisis?,"Hipotensi Intradialisis, Kram otot, nyeri dada...",Kepmenkes No. HK.01.07/MENKES/1634/2023 tentan...,teks,antibiotika empiris sampai ada hasil kultur da...,"A, terjadi reaksi hipersensitivitas terhadap k...",Intralipid 20%). \n \n7) Komplikasi Akut Hemod...,"\nbe given vasopressors like dopamine, dobutam..."


In [20]:
excel_pertanyaan["Nutrisi"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa saja parameter penilaian status nutrisi da...,"Parameter penilaian meliputi asupan makanan, p...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,rekomendasi terapi nutrisi pada anak dengan P...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,Rekomendasi parameter dan frekuensi penilaian...,"\nIn summary, the recommended parameters for a..."
2,Apa tujuan terapi nutrisi pada anak dengan PGK?,Tujuan terapi nutrisi pada anak dengan PGK ada...,teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,PENDAHULUAN Penyakit ginjal kronik (PGK) adala...,rekomendasi terapi nutrisi pada anak dengan P...,\nJawaban:\nTujuan terapi nutrisi bagi anak de...
3,apa yang menyebabkan gangguan selera makan pad...,"Pada anak dengan poliuria, rasa haus lebih nya...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,d. Kekurangan berat badan lebih dari 10% berat...,"pada anak dengan PGK derajat 2-5 dan 5D, peraw...",Tujuan terapi nutrisi ini dapat dicapai dengan...,\nthe National Kidney Foundation published gui...
4,bagaimana gambaran penyakit ginjal kronik dera...,Kerusakan ginjal dengan LFG normal atau mening...,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Klasifikasi derajat penyakit ginjal kronik.Der...,PENDAHULUAN Penyakit ginjal kronik (PGK) adala...,beberapa faktor yang menyebabkan kerusakan\nbe...,\nPenghasilan gambar penyakit ginjal kronik de...
5,berapa interval penilaian asupan makanan anak ...,1 - 3 bulan,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi parameter dan frekuensi penilaian...,Rekomendasi asupan makanan yang adekuat pada ...,rekomendasi terapi nutrisi pada anak dengan P...,### Answer:\nThe interval for nutritional asse...
6,berapa rekomendasi asupan asam pantotenat untu...,4 mg/hr,tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi asupan makanan yang adekuat pada ...,Rekomendasi asupan protein pada anak PGK dera...,. Rekomendasi KDOQI untuk asupan fosfat pada a...,Answer:\nThe recommended daily ascorbic acid o...
7,apa penyebab kekurangan vitamin D pada anak de...,"Gaya hidup dengan aktivitas fisik rendah, sehi...",teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,b. Vitamin D Pada anak dengan PGK terdapat def...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,"Pada anak PGK, kondisi hiperfosfatemia atau hi...",\nAnswer:\nThe causes of vitamin D deficiency ...
8,berapa rekomendasi asupan protein untuk pasien...,"0,95-1,95 g/kg/hari",tabel,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,Rekomendasi asupan protein pada anak PGK dera...,. Asupan protein untuk anak PGK berdasarkan DR...,penyebab 25% kematian dan kesakitan pada anak...,\nUntuk anak dengan penyakit ginjal kronis der...
9,apakah pasien anak PGK perlu mengurangi asupan...,Pada anak pengidap PGK derajat 3-5 dengan olig...,teks,Nutrisi pada Anak dengan Penyakit Ginjal Kroni...,bermanfaat mencegah dan menangani hiperpar...,"Pada pasien dewasa dengan PGK 3-5D, KDOQI meny...",ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,\nJawaban:\nChildren with PGK mungkin perlu me...


In [21]:
excel_pertanyaan["tatalaksana pada anak"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,apa faktor yang menyebabkan kerusakan berkelan...,"glomerulosklerosis, pembentukan fibrosis tubul...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",beberapa faktor yang menyebabkan kerusakan\nbe...,abnormalitas struktur atau fungsi ginjal denga...,menunjukkan hematuria dan proteinuria pada ana...,"\nsymptoms of uremia (lethargy, pericarditis t..."
2,apa penyebab umum penyakit ginjal kronis pada ...,abnormalitas kongenital seperti hipoplasia ata...,teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",abnormalitas struktur atau fungsi ginjal denga...,ABSTRAK Penyakit ginjal kronik (PGK) merupakan...,Penyakit ginjal kronis stadium 5 merupakan ind...,\nJawaban:\nPenyebab umum penyakit ginjal kron...
3,apa peran ACE inhibitors dalam penanganan hipe...,melindungi nefron yang tersisa dari cedera leb...,teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",menunjukkan hematuria dan proteinuria pada ana...,"Inhibitor atau ARB \n4)Jika HbA1C <7%, tekanan...",3. Pencegahan Progresivitas Penyakit Ginjal Kr...,\nApapun peran ACE inhibitors dalam penanganan...
4,berapa kebutuhan protein harian anak 3-10 tahu...,"4-6 tahun: 1,2 g/kg BB\n7-10 tahun: 1,0 g/kg BB",tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Rekomendasi Nutrisi Bagi Anak Dengan Penyakit ...,. Asupan protein untuk anak PGK berdasarkan DR...,ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,\nAnswer :\nThe daily protein requirements for...
5,berapa dosis EPO pada pasien PGK dengan anemia?,"Dosis biasa EPO adalah 300 unit/kg, dibagi dal...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",heparin konstan 250-2000 unit/jam \n(biasanya ...,(2)Injeksi subkutan untuk pasien anak dengan\n...,Kebutuhan elektrolit harian untuk natrium dan ...,\nAnswer:\nThe specific EPO dosage for patient...
6,bagaimana penanganan dislipidemia pada anak de...,"intervensi gaya hidup, seperti latihan sedang,...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Hipertrigliserida berperan penting dalam muncu...,"normal, tidak dianjurkan melakukan intervensi ...",kenaikan atau kehilangan berat badan. b. Tamba...,\nthe leading cause of death in patients with ...
7,apa kontraindikasi absolut untuk transplantasi...,"keganasan aktif, terutama jika telah bermetast...",teks,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",1) Indikasi dan kontraindikasi resipien transp...,Penyakit ginjal kronis stadium 5 merupakan ind...,3) Indikasi dan kontraindikasi donor transplan...,### Response:\nAbsolute contraindications for ...
8,apa yang bisa dilakukan untuk menangani Hipovo...,pemulihan cairan ekstraselular dengan natrium ...,tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",Penanganan gangguan elektrolitHiponatremia (ko...,Rekomendasi 10: \nTatalaksana penyebab PGTA y...,Tatalaksana hipotensi intradialisis disesuaika...,### Response:\nTo manage hipovolemia with hypo...
9,bagaimana pemberian obat vasolidator langsung ...,"Hidralazin, Dosis oral awal 1-2 mg/kg, Dosis o...",tabel,"Tatalaksana Penyakit Ginjal Kronik pada Anak, ...",e.Hipertensi\nAngka kejadian hipertensi pada a...,stenosis arteri renalis atau penyempitan pembu...,. Dosis Obat Hipertensi Pada AnakDiuretikaHidr...,"\nAce Inhibitor Lisinopril, Oral doses Initial..."


In [22]:
excel_pertanyaan["tatalaksana anak dan bayi"]

,Pertanyaan,Jawaban,tipe sumber,Dokumen,Dokumen 1,Dokumen 2,Dokumen 3,Generated Answer
No,,,,,,,,
1,Apa saja indikasi absolut untuk memulai dialis...,"Indikasi absolut meliputi anuria, gangguan ele...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",ABSTRAK Penyakit ginjal kronik (PGK) merupakan...,beratnya kurang dari 10 kg. Sebuah studi Erop...,"membutuhkan dialisis (CKD5d), hampir dua kali...",\nAnswer:\nAbsolute indications for starting d...
2,Apa saja manifestasi klinis yang sering dijump...,Manifestasi klinis meliputi gangguan pertumbuh...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",dominan. Penyebab utama PGK pada anak ada...,"akibat infeksi atau kelainan yang didapat, di ...",Pemberian terapi pengikat kalium seperti polys...,\nAnswer:\nManifestations commonly observed in...
3,Apa perbedaan penting antara penyakit ginjal t...,"Perbedaan penting adalah etiologi PGK, dimana ...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...","KDOQI) pada tahun 2002, merupakan penyakit gi...",dominan. Penyebab utama PGK pada anak ada...,"akibat infeksi atau kelainan yang didapat, di ...","\nIn conclusion, there are several key differe..."
4,Apa dampak PGK pada pertumbuhan anak?,Gangguan pertumbuhan yang meningkat saat LFG m...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",ABSTRACT Penyakit ginjal kronik (PGK) adalah k...,4. Kortikosteroid Penggunaan kortikosteroid da...,"akibat infeksi atau kelainan yang didapat, di ...",\nPGK or chronic kidney disease affects the gr...
5,apa yang memengaruhi keberhasilan hemodialisis...,akses vaskular yang baik: fistula arteriovenos...,teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",antibiotika empiris sampai ada hasil kultur da...,"sisi lain, ada penurunan risiko hipotensi int...",Intralipid 20%). \n \n7) Komplikasi Akut Hemod...,\nbe administered intravenously if necessary.\...
6,apa anti koagulan yang paling sering digunakan...,Unfractionated heparin (UFH),teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",Untuk kebanyakan bayi dan anak-anak dengan be...,sebaiknya dikerjakan jika dicurigai terdapat i...,(HDx). Hasil penelitian tersebut menunjukkan b...,\nUnfractionated heparin (UFH) is the most com...
7,apa gejala Sindrom disekuilibrium?,"nyeri kepala atau kejang, muntah, kelelahan, m...",teks,"Tatalaksana Hemodialisis pada Anak dan Bayi, H...",insersi jarum. Persiapan psikologis anak dan ...,pasien risiko tinggi (terutama bila urea >60 m...,"(3)elektrolit (kalium, natrium, klorida, kalsi...",\nDisequilibrium Syndrome is a complication th...
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
with pd.ExcelWriter(f'pertanyaan benchmark_{embed_model}_{loader}_{reranker}.xlsx') as writer:  
    excel_pertanyaan["PNPK"].to_excel(writer, sheet_name='PNPK')
    excel_pertanyaan["Nutrisi"].to_excel(writer, sheet_name='Nutrisi')
    excel_pertanyaan["tatalaksana pada anak"].to_excel(writer, sheet_name='tatalaksana pada anak')
    excel_pertanyaan["tatalaksana anak dan bayi"].to_excel(writer, sheet_name='tatalaksana anak dan bayi')